# Simulation des nombre aléatoire

1 ere partie: la génération des nombres aléatoires qui suit la loi exponentielle pour les utiliser dans la simulation d'un système d'attente M/M/S

In [1]:
!pip install simpy
import math

distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 20.1b1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


Je vais utiliser le GCL pour générer des nombres aléatoires qui suivent la loi uniforme, la fonction GCL permet aux utilisateurs de paramétrer le générateur, pour le reste de ce Tp je vais utiliser la version de GCL utilise par le GCC qui a comme paramètres : 
    a=1103515245 
    c=12345
    m=(2)^31=2147483648

In [2]:
def GCL(m, a, c, seed):
   while True:
        seed = (a * seed + c) % m
        yield seed
def GCL_GCC(seed):
    while True:
        seed = (1103515245 * seed + 12345) % 2147483648
        yield seed

Pour avoir des nombres entre 0 et 1 je vais diviser les résultats de la fonction GCL_GCC par m=2147483648

In [3]:
def uniform_0_1(seed): 
    for i in GCL_GCC(seed):
        yield i/2147483648

Exemples des résultats des fonctions précédentes avec seed = 10 : 

In [4]:
seed=10
j=0
print("suite de 5 nombres aléatoires par le générateur de GCC")
for i in GCL_GCC(seed):
    if(j==5):
        break
    j+=1
    print(i, end=" ")
print("\n")
print("suite de 5 nombres aléatoires qui suit la loi uniforme entre 0 et 1 ")
j=0
for i in uniform_0_1(seed):
    if(j==5):
        break
    j+=1
    print(float("{0:.2f}".format(i)), end=" ")

suite de 5 nombres aléatoires par le générateur de GCC
297746555 1849040536 736986865 581309142 1106733399 

suite de 5 nombres aléatoires qui suit la loi uniforme entre 0 et 1 
0.14 0.86 0.34 0.27 0.52 

pour la génération des nombres aléatoires qui suivent la loi exponentielle j'utilise la méthode de fonction inverse vue en cours qui est :
1- Générer ܷU ↝ ܷU[0,1]
2- calculer X= -(1/λ)log(U) ~ EXP(λ)

In [5]:
def Exp(λ,seed): 
    for i in uniform_0_1(seed):
        yield -(1/λ) * math.log(i)

Exemple d'une suite de 5 nombres aléatoire qui suivent la loi exponentielle générée par la fonction EXP avec seed= 3 et λ = 0.1:

In [6]:
seed = 3
λ = 0.1
j=0
print("Suite de 5 nombre aléatoire qui suit la loi exponentiel de parametre λ = 0.1")
for i in Exp(λ,seed):
    if(j==5):
        break
    j+=1
    print(float("{0:.2f}".format(i)), end=" ")

Suite de 5 nombre aléatoire qui suit la loi exponentiel de parametre λ = 0.1
6.13 15.28 11.51 13.72 4.63 

2éme partie : simulation d'un systeme d'attente M/M/S 
    

Pour la simulation je vais utiliser le package Simpy qui offre un environnement d'exécution des tâches "processus" (services dans notre cas)qui s'exécutent sur des machines "ressources" (serveurs dans notre cas) à travers des  évènement "events" (l'arrivée et le départ des clients dans notre cas")

In [25]:
from statistics import mean
import simpy

je vais specifier :
1- le seed
2- le nombre de serveurs S 
3- nombre initial de clients dans la fil Nmbr_Q

In [26]:
SEED = 7
nmbr_serveurs =6
temps_de_simulation =100
Nmbr_Q =0 
historique_de_systeme = []
temps_de_service = []
temps_de_sejour =[]

je vais créer un class système qui contient deux fonctions, l'initialisation du système avec le nombre de machines specifier et le processus servir qui consiste à consommer un temps à chaque service ce temps suit la loi exponentiel de paramètre µ qu'on va specifier plus tard

In [27]:
class Systeme(object):
    Nmbr_Q = 0 
    def __init__(self, env, nmbr_serveurs):
        self.env = env
        self.serveur = simpy.Resource(env, nmbr_serveurs)

    def servir(self, client,temps_service_unique):
        yield self.env.timeout(temps_service_unique)

un autre processus pour l'arrivée, l'attent de service et le départ de client

In [28]:
def client(env, nom, sys,temps_service_unique,temps_arrive):
    global Nmbr_Q
    Nmbr_Q= Nmbr_Q+1
    historique_de_systeme.append(Nmbr_Q)
    with sys.serveur.request() as request:
        yield request
        Nmbr_Q =Nmbr_Q - 1
        historique_de_systeme.append(Nmbr_Q)
        yield env.process(sys.servir(nom,temps_service_unique))
        print('%s quitte le système à %.2f.' % (nom, env.now))
        temps_de_sejour.append(env.now-temps_arrive)

Le dernier processus est le processus de création de nouveaux clients pendant que le système fonctionne le temps d'arriver des clients suit la loi de poisson de paramètre λ qu'on va specifier plus tard donc le temps d'inter arrive des clients suit la loi exponentielle de paramètre λ.

In [29]:
def setup(env, nmbr_serveurs):
    #creation de systeme
    systeme = Systeme(env, nmbr_serveurs)
    # creation de nouveaux clients pendant que le système fonctionne
    j =0
    expo_iterator=Exp(µ,SEED)
    while True:
        for i in Exp(λ,SEED):
            yield env.timeout(float("{0:.2f}".format(i)))
            temps_arrive = env.now
            j += 1
            nom ='client %d' % j
            temps_service_unique=next(expo_iterator)
            temps_de_service.append(temps_service_unique)
            print('%s arrive au system à %.2f.' % (nom, temps_arrive))
            env.process(client(env, 'client %d' % j, systeme,temps_service_unique,temps_arrive))

# Attention! ,il faut saisir les paramètres dans la cellule suivante

In [30]:
SEED = int(input("seed: >>>"))
λ = float(input("λ: >>>"))
µ = float(input("µ: >>>"))

seed: >>>6
λ: >>>6
µ: >>>1


Création d'un environnement et démarrage de processus setup(création de nouveaux clients pendant que le système fonctionne)

In [31]:
env = simpy.Environment()
env.process(setup(env, nmbr_serveurs))

<Process(setup) object at 0x258e354ee10>

Lancement de la simulation 

In [32]:
env.run(until=temps_de_simulation)

client 1 arrive au system à 0.41.
client 2 arrive au system à 0.45.
client 3 arrive au system à 0.64.
client 4 arrive au system à 0.67.
client 2 quitte le système à 0.70.
client 4 quitte le système à 0.85.
client 5 arrive au system à 0.98.
client 6 arrive au system à 1.11.
client 7 arrive au system à 1.15.
client 8 arrive au system à 1.33.
client 7 quitte le système à 1.42.
client 9 arrive au system à 1.68.
client 3 quitte le système à 1.76.
client 10 arrive au system à 1.81.
client 11 arrive au system à 1.89.
client 6 quitte le système à 1.91.
client 12 arrive au system à 2.15.
client 13 arrive au system à 2.19.
client 14 arrive au system à 2.24.
client 15 arrive au system à 2.30.
client 11 quitte le système à 2.37.
client 8 quitte le système à 2.39.
client 16 arrive au system à 2.52.
client 10 quitte le système à 2.58.
client 17 arrive au system à 2.59.
client 18 arrive au system à 2.61.
client 13 quitte le système à 2.61.
client 19 arrive au system à 2.82.
client 20 arrive au system

client 595 arrive au system à 98.33.
client 596 arrive au system à 98.35.
client 583 quitte le système à 98.39.
client 584 quitte le système à 98.59.
client 597 arrive au system à 98.59.
client 598 arrive au system à 98.61.
client 570 quitte le système à 98.66.
client 582 quitte le système à 98.68.
client 599 arrive au system à 98.83.
client 578 quitte le système à 98.87.
client 585 quitte le système à 98.89.
client 600 arrive au system à 98.92.
client 601 arrive au system à 98.95.
client 602 arrive au system à 99.13.
client 603 arrive au system à 99.16.
client 579 quitte le système à 99.27.
client 590 quitte le système à 99.32.
client 587 quitte le système à 99.43.
client 591 quitte le système à 99.49.
client 572 quitte le système à 99.54.
client 604 arrive au system à 99.66.
client 594 quitte le système à 99.68.
client 593 quitte le système à 99.74.
client 596 quitte le système à 99.85.
client 595 quitte le système à 99.97.
client 605 arrive au system à 99.97.


3èmè partie : Comparaison des carectiristiques de systeme sur le plan theorique avec les resultats obtenue d'apres la simulation que j'ai fait 

# Remarque : 
pour le calcul des caractéristiques du système sur le plan théorique je vais utiliser le code suivant de monsieur ZOUAHI Hafidh

In [33]:
from math import factorial
class QueuingSystem:
    def __init__(self, λ, µ, s, l=None):
        self.s = s
        self.λ = λ
        self.µ = µ
        self.ρ = (λ/(s*µ))
        self.l = None
        if l is not None:
            self.l = l
        if l is None:
            self.p_zero = (sum((1/factorial(k))*(λ/µ)**k for k in range(s))\
                           + (1/factorial(s))*((λ/µ)**s)*((s*µ)/(s*µ - λ))\
                          )**(-1)
        else:
            u = λ / µ
            if u == s:
                self.p_zero = (sum(s**i/factorial(i) for i in range(s))\
                               + (s**s/factorial(s))*(l - s + 1)
                              )**(-1)
            else:
                self.p_zero = (sum(u**i/factorial(i) for i in range(s))\
                               + ((u**s)/factorial(s))*((1 - (u/s)**(l-s+1))/(1-u/s))
                              )**(-1)
    
    def isStationary(self):
        return self.ρ < 1
    
    def computePi(self, i):
        u = self.λ / self.µ
        if i <= self.s:
            if self.l is None:
                return (self.λ**i/(factorial(i)*self.µ**i))*self.p_zero
            else:
                return (u**i/factorial(i))*self.p_zero
        else:
            if self.l is None:
                return (self.λ**i/(self.s**(i-self.s)*factorial(self.s)*µ**i))*self.p_zero
            else:
                return (u**i/(factorial(self.s)*s**(i-s)))*self.p_zero
    
    def computeNfBar(self):
        if self.l is None:
            self.nf_bar = (self.ρ*self.computePi(self.s))/(1-self.ρ)**2
        else:
            if self.ρ == 1:
                self.nf_bar = self.p_zero*((self.s**self.s)/factorial(s))*((self.l - self.s)(self.l - self.s + 1)/2)
            else:
                self.nf_bar = self.p_zero*(((self.s*self.ρ)**self.s*self.ρ)/(factorial(self.s)*(1-self.ρ)**2))*(1-self.ρ**(self.l-self.s+1)-(1-self.ρ)*(self.l-self.s+1)*self.ρ**(self.l-self.s))
        return self.nf_bar
    
    def computeNsBar(self):
        if self.l is None:
            self.ns_bar = self.nf_bar + self.s*self.ρ
        else:
            pl = self.computePi(self.l)
            self.ns_bar = self.nf_bar + (self.λ*(1 - pl))/self.µ
        return     self.ns_bar
        
    def computeTfBar(self):
        if self.l is None:
            self.tf_bar = self.nf_bar / self.λ
        else:
            pl = self.computePi(self.l)
            self.tf_bar = self.nf_bar / (self.λ*(1-pl))
        return self.tf_bar
        
    def computeTsBar(self):
        self.ts_bar = self.tf_bar + 1 / self.µ
        return self.ts_bar
s = nmbr_serveurs 
system = QueuingSystem(λ, µ, s)
if system.isStationary():
    print(">>> There you go: ")
    print("[+] nf-bar =", system.computeNfBar())
    print("[+] ns-bar =", system.computeNsBar())
    print("[+] tf-bar =", system.computeTfBar())
    print("[+] ts-bar =", system.computeTsBar())
else:
    print("[-] Not stationary")

ZeroDivisionError: float division by zero

Calcule des caractéristiques de simulation :

In [34]:
print("NS-bar = ",mean(historique_de_systeme))

NS-bar =  5.253532834580216


Temps de service moyen : 

In [35]:
temps_dans_la_file =[]
for i in range(len(temps_de_sejour)-1):
    temps_dans_la_file.append(temps_de_sejour[i]-temps_de_service[i])
print("temps d'attente moyen = ",float("{0:.2f}".format(mean(temps_dans_la_file))))

temps d'attente moyen =  0.77


In [36]:
print("Ts-bar= ",float("{0:.2f}".format(mean(temps_de_sejour))))

Ts-bar=  1.76
